#Libraries 
All needed libraries to use

In [ ]:
#LIBRERIES

import numpy as np
import os
from PIL import Image, ImageFile
from random import random

import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, AveragePooling2D, Concatenate, Multiply, ConvLSTM2D, Lambda, Dropout, Flatten, Dense
from tensorflow.keras import layers, Model, regularizers, activations
from keras.initializers import Constant
from keras.constraints import unit_norm, non_neg
from keras import backend as K
from keras.engine.base_layer import Layer
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import AUC, Precision, Recall

from keras_preprocessing import image
from keras_preprocessing.image import ImageDataGenerator

import matplotlib.pyplot as plt
import matplotlib.patheffects as path_effects

import cv2

In [ ]:
#MOUNT GOOGLE DRIVE 
from google.colab import drive
drive.mount('/content/drive')

#Useful functions
All functions used to deal with data (e.g. load files, define dataset, plot images ad history of the model, etc.)

In [ ]:
#USEFUL FUNCTIONS

#Get a list of paths of images from files
def getFiles(train, path):
    images = []
    count = 0
    for file in  sorted(os.listdir(path +'/')):
        images.append(path + '/'+  file)
    return images

#Read the images from path
def readImage(img_path,size):

  img = cv2.imread(img_path) 
  img = cv2.resize(img,(size,size))

  return img

#Read the mask (image labels) from path
def readMask(img_path,size):

  img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE) 
  img = cv2.resize(img,(size,size))
  img = img.reshape((size,size,1))

  return img

#Get a list of array of images (or mask) from paths
def list_from_files(files, img_size, isMask = False):

  l = []

  if(isMask):

    for mask_path in files:
      l.append(readMask(mask_path, img_size))

  else:

    for img_path in files:
      l.append(readImage(img_path, img_size))

  return l

#Prepare train, validation and test data
def dataset(X_train, y_train, train=0.7, validation=0.2, test=0.1):

  X_train = np.asarray(X_train)
  y_train = np.asarray(y_train)

  dim = X_train.shape[0]
  n_train = np.int(dim*train)
  n_validation = np.int(dim*validation)
  n_test = dim - n_train - n_validation

  x = X_train[0:n_train]
  y = y_train[0:n_train]
  x = tf.convert_to_tensor(x, dtype=tf.float32)
  y = tf.convert_to_tensor(y, dtype=tf.float32)

  x_val = X_train[n_train:n_train+n_validation]
  y_val = y_train[n_train:n_train+n_validation]
  x_val = tf.convert_to_tensor(x_val, dtype=tf.float32)
  y_val = tf.convert_to_tensor(y_val, dtype=tf.float32)

  x_test = X_train[-n_test:]
  y_test = y_train[-n_test:]
  x_test = tf.convert_to_tensor(x_test, dtype=tf.float32)
  y_test = tf.convert_to_tensor(y_test, dtype=tf.float32)

  print("Using: "+np.str(n_train)+" images for training")
  print("Using: "+np.str(n_validation)+" images for validation")
  print("Using: "+np.str(n_test)+" images for test")
  print("For a total of "+np.str(dim)+" images found.")

  return x, y, x_val, y_val, x_test, y_test

#Predict the mask from an image
def predict_mask(model, img_size, train_files, n):

  im = readImage(train_files[n], img_size)
  im = im.reshape([1, img_size, img_size, 3])
  im = tf.convert_to_tensor(im, dtype=tf.float32)
  new = model.predict(im)
  return np.asarray(new)

#Plot images, corresponding mask and predicted mask
def plot_samples_predictions(model, x, y, train_files, img_size, n_samples=5):

  fig1, imgs = plt.subplots(n_samples, 3, figsize=(15, 15), sharex=True, sharey=True)

  for i in range(n_samples):

    n = np.random.randint(0,len(x))
  
    mask = predict_mask(model, img_size, train_files, n)

    imgs[i,0].set_title('Image Test')
    imgs[i,0].imshow(np.asarray(x[n]), cmap=plt.cm.gray)

    imgs[i,1].set_title('Ground Truth')
    imgs[i,1].imshow(np.asarray(y[n]).squeeze(), cmap=plt.cm.gray)

    imgs[i,2].set_title('Prediction')
    imgs[i,2].imshow(mask.squeeze(), cmap=plt.cm.gray)

  plt.show()

#Plot curves of history of training of model (loss, AUC, precision, recall)
def plot_history(H):

  k = []
  for i in H.history.keys():
    k.append(i)

  loss = k[0]
  auc = k[1]
  precision = k[2]
  recall = k[3]
  val_loss = k[4]
  val_auc = k[5]
  val_precision = k[6]
  val_recall = k[7]

  fig, (ax1, ax2) = plt.subplots(2, 2,figsize=(15, 10))

  ax1[0].set_title('TRANINING LOSS')
  ax1[0].set(xlabel='Epoch', ylabel='Loss')
  ax1[0].plot(H.history[loss], label='loss')
  ax1[0].legend(loc="upper left")

  ax1[1].set_title('TRANINING METRICS')
  ax1[1].set(xlabel='Epoch', ylabel='Metrics')
  ax1[1].plot(H.history[auc], label='AUC')
  ax1[1].plot(H.history[precision], label='precision')
  ax1[1].plot(H.history[recall], label='recall')
  ax1[1].legend(loc="upper left")

  ax2[0].set_title("VALIDATION LOSS")
  ax2[0].set(xlabel='Epoch', ylabel='Loss')
  ax2[0].plot(H.history[val_loss], label='loss')
  ax2[0].legend(loc="upper left")

  ax2[1].set_title('VALIDATION METRICS')
  ax2[1].set(xlabel='Epoch', ylabel='Metrics')
  ax2[1].plot(H.history[val_auc], label='AUC')
  ax2[1].plot(H.history[val_precision], label='precision')
  ax2[1].plot(H.history[val_recall], label='recall')
  ax2[1].legend(loc="upper left")

  plt.show()

#Plot history (only with loss and accuracy)
def plot_history2(H):

  k = []
  for i in H.history.keys():
    k.append(i)

  loss = k[0]
  acc = k[1]
  val_loss = k[2]
  val_acc = k[3]

  fig, (ax1, ax2) = plt.subplots(2, 2,figsize=(15, 10))

  ax1[0].set_title('TRANINING LOSS')
  ax1[0].set(xlabel='Epoch', ylabel='Loss')
  ax1[0].plot(H.history[loss], label='loss')
  ax1[0].legend(loc="upper left")

  ax1[1].set_title('TRANINING METRICS')
  ax1[1].set(xlabel='Epoch', ylabel='Metrics')
  ax1[1].plot(H.history[acc], label='Accuracy')
  ax1[1].legend(loc="upper left")

  ax2[0].set_title("VALIDATION LOSS")
  ax2[0].set(xlabel='Epoch', ylabel='Loss')
  ax2[0].plot(H.history[val_loss], label='loss')
  ax2[0].legend(loc="upper left")

  ax2[1].set_title('VALIDATION METRICS')
  ax2[1].set(xlabel='Epoch', ylabel='Metrics')
  ax2[1].plot(H.history[val_acc], label='Accuracy')
  ax2[1].legend(loc="upper left")

  plt.show()

#print evaluation of model on test data
def print_model_evaluation(model):

  loss, auc, prec, rec = mantranet.evaluate(x_test,y_test)
  print("Loss: ", loss)
  print("AUC: ", auc)
  print("Precision: ", prec)
  print("Recall: ", rec)

#Classes used in the neural network model

In [ ]:
class BayarConstraint(tf.keras.constraints.Constraint):

  def __init__(self):
    return

  def call(self, w):
    rows, cols, _, _ = K.int_shape(w)
    w[rows//2,cols//2] = 0
    sum = K.sum(w, axis=(0,1), keepdims=True)
    w /= sum
    w[rows//2,cols//2] = -1
    return w

In [ ]:
class BayarConv2D(Layer):

  def __init__(self, filters, kernel_size, padding='SAME', activation=None, kernel_initializer='glorot_uniform',
               kernel_regularizer=None, kernel_constraint=BayarConstraint(), trainable=True, name="BayarConv2D"): 
    super(BayarConv2D, self).__init__(trainable=trainable, name=name)  
    self.filters = filters
    self.kernel_size = kernel_size
    self.kernel_initializer = kernel_initializer
    self.kernel_regularizer = kernel_regularizer
    self.padding = padding
    self.activation = activation
    self.kernel_constraint = kernel_constraint

  def build(self, input_shape):
    input_shape = tf.TensorShape(input_shape)
    input_channel = 3
    kernel_shape = self.kernel_size + (input_channel, self.filters)
    self.kernel = self.add_weight(name='kernel',
                                  shape=kernel_shape,
                                  initializer=self.kernel_initializer,
                                  regularizer=self.kernel_regularizer,
                                  constraint=self.kernel_constraint,
                                  trainable=True,
                                  dtype=self.dtype)
    self.built = True

  def get_kernel(self):
    return self.kernel

  def call(self, inputs):
    outputs = tf.nn.convolution(inputs, self.kernel, padding=self.padding)
    if self.activation is not None:
      act = layers.Activation(self.activation)
      return act(outputs)
  
    return self.kernel

In [ ]:
class SRMConv2D(Layer):

  def __init__(self, filters = 3, kernel_size = (5,5), padding='SAME', activation="relu",
               kernel_initializer='glorot_uniform', trainable=True, name="SRMConv2D"):
    super(SRMConv2D, self).__init__(trainable=trainable, name=name)   
    self.filters = filters  
    self.padding = padding
    self.activation = activation  
    self.kernel = []

    #srm kernel 1
    self.srm1 = np.zeros([5,5]).astype('float32')
    self.srm1[1:-1,1:-1] = np.array([[-1, 2, -1],
                                  [2, -4, 2],
                                  [-1, 2, -1]] )
    self.srm1 /= 4.

    #srm kernel 2                                                                                                                                
    self.srm2 = np.array([[-1, 2, -2, 2, -1],
                        [2, -6, 8, -6, 2],
                        [-2, 8, -12, 8, -2],
                        [2, -6, 8, -6, 2],
                        [-1, 2, -2, 2, -1]]).astype('float32')
    self.srm2 /= 12.

    # srm kernel 3                                                                                                                                
    self.srm3 = np.zeros([5,5]).astype('float32')
    self.srm3[2,1:-1] = np.array([1,-2,1])
    self.srm3 /= 2.

  def build(self, input_shape):
    kernel = []
    srm_list = [self.srm1, self.srm2, self.srm3]
    for idx, srm in enumerate(srm_list):
      for ch in range(3) :
        this_ch_kernel = np.zeros([5,5,3]).astype('float32')
        this_ch_kernel[:,:,ch] = srm
        kernel.append( this_ch_kernel )
    kernel = np.stack( kernel, axis=-1 )
    self.kernel = K.variable(kernel, dtype='float32', name='srm')

    self.built = True

  def get_kernel(self):
    return self.kernel

  def call(self, inputs):  
    outputs = tf.nn.convolution(inputs, self.kernel, padding=self.padding)
    if self.activation is not None:
      return self.activation(outputs)
    
    return self.kernel

In [ ]:
class GlobalStd2D( Layer ) :

    def __init__(self, min_std_val=1e-5, name="GlobalStd2D"):
        self.min_std_val = min_std_val
        super(GlobalStd2D, self ).__init__()

    def build( self, input_shape ) :
        feats = input_shape[-1]
        std_shape = (1,1,1, feats)
        self.min_std = self.add_weight( shape=std_shape,
                                        initializer=Constant(self.min_std_val),
                                        name='min_std',
                                        constraint=non_neg())
        self.built = True

    def call( self, x ) :
        x_std = K.std( x, axis=(1,2), keepdims=True )
        x_std = K.maximum( x_std, self.min_std_val/10. + self.min_std )
        return x_std

In [ ]:
class ZPool2D(Layer):

  def __init__(self, win_size, filters = None, name="ZPool2D"):  
    super(ZPool2D, self).__init__(name=name)
    self.win_size = win_size

  def call(self, input, globalPool=False):
    avgs = []
    for dim in self.win_size:

      avg = AveragePooling2D(pool_size=(dim,dim), strides=(1, 1), data_format='channels_last', padding='same')
      mu = avg(input)
      d = input-mu
      avgs.append(d)

    if(globalPool==True):
      mu = K.mean( input, axis=(1,2), keepdims=True )
      d = input-mu
      avgs.append(d) 

    avgs = K.stack(avgs, axis=1)

    return avgs

#Hyperparameters to use in the model

In [ ]:
#HYPERPARAMETERS

IMG_SIZE = 90
EPOCHS = 100
batch_size = 64
learning_rate = 0.0001
pad = "same"
opt = Adam(learning_rate=learning_rate)
loss_f = "categorical_crossentropy"
metrics = [tf.keras.metrics.AUC(), tf.keras.metrics.Precision(), tf.keras.metrics.Recall()]
metrics_imc = "accuracy"

#Load and prepare data

In [ ]:
#DATASET PATH
casia_imgs_path = "/content/drive/MyDrive/myDataset/CM"
casia_mask_path = "/content/drive/MyDrive/myDataset/GT"

#LOAD DATA
imgs_train_files = getFiles(False, casia_imgs_path)
mask_train_files = getFiles(False, casia_mask_path)
imgs_list = list_from_files(imgs_train_files, IMG_SIZE)
mask_list = list_from_files(imgs_train_files, IMG_SIZE, isMask=True)

#ORGANIZE DATASET FOR MANTRANET TASK
x_train, y_train, x_val, y_val, x_test, y_test = dataset(imgs_list, mask_list)
print("Shape of trainig: "+np.str(x_train.shape)+" - "+np.str(y_train.shape))
print("Shape of validation: "+np.str(x_val.shape)+" - "+np.str(y_val.shape))
print("Shape of test: "+np.str(x_test.shape)+" - "+np.str(y_test.shape))

#ORGANIZE DATASET FOR IMC TASK
path_imc = "/content/drive/MyDrive/myIMCdataset"

train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
train_imc = train_datagen.flow_from_directory(path_imc, target_size=(IMG_SIZE, IMG_SIZE),
                                          batch_size=batch_size, class_mode='categorical', subset='training')
val_imc = train_datagen.flow_from_directory(path_imc, target_size=(IMG_SIZE, IMG_SIZE),
                                         batch_size=batch_size, class_mode='categorical', subset='validation')

test_datagen = ImageDataGenerator(rescale=1./255)
test_imc = test_datagen.flow_from_directory(path_imc, target_size=(IMG_SIZE, IMG_SIZE),
                                         batch_size=batch_size, class_mode='categorical')

#First implementation of the model in a class

Here all layers are implemented in a class building a single model

In [ ]:
#IMPLEMENTATION OF THE MODEL IN A CLASS

class ManTraNet(Model):

  def __init__(self):
    super(ManTraNet, self).__init__()
    
    self.conv1 = Conv2D(10, (5,5), input_shape=(IMG_SIZE, IMG_SIZE, 3), padding=pad, activation="relu")
    self.bayar_conv = BayarConv2D(3, (5,5), activation=activations.relu)
    self.srm_conv = SRMConv2D(activation=activations.relu)

    self.conv2 = Conv2D(32, (3,3), padding=pad, activation="relu")
    self.conv3 = Conv2D(64, (3,3), padding=pad, activation="relu")
    self.conv4 = Conv2D(64, (3,3), padding=pad, activation="relu")
    self.conv5 = Conv2D(128, (3,3), padding=pad, activation="relu")
    self.conv6 = Conv2D(128, (3,3), padding=pad, activation="relu")
    self.conv7 = Conv2D(128, (3,3), padding=pad, activation="relu")
    self.conv8 = Conv2D(256, (3,3), padding=pad, activation="relu")
    self.conv9 = Conv2D(256, (3,3), padding=pad, activation="relu")
    self.conv10 = Conv2D(256, (3,3), padding=pad, activation="relu")
    self.conv11 = Conv2D(256, (3,3), padding=pad, activation="relu")
    self.conv12 = Conv2D(256, (3,3), padding=pad, activation="relu")
    self.conv13 = Conv2D(256, (3,3), padding=pad, activation=None, activity_regularizer=regularizers.l2(1e-5))

    self.conv14 = Conv2D(64, (1,1), activation=None, use_bias=False, kernel_constraint = unit_norm( axis=-2 ), padding=pad)
    self.bn = BatchNormalization(axis=-1, center=True, scale=True, name="bn1")

    self.conv15 = Conv2D(64, (3,3), padding=pad, activation="relu")
    self.conv16 = Conv2D(64, (3,3), padding=pad, activation="relu")
    self.bn2 = BatchNormalization(axis=-1, center=True, scale=True, name="bn2")

    self.globalstd = GlobalStd2D()
    self.ZPool = ZPool2D(win_size=[7,15,31])
    self.conv_lstm = ConvLSTM2D(8, (7,7), activation='tanh', recurrent_activation='hard_sigmoid',
                                padding='same', name='cLSTM', return_sequences=False)
    
    self.conv_last = Conv2D(1, (7,7), padding=pad, activation="sigmoid")

  def call(self, input):

    x1 = self.conv1(input)
    x2 = self.bayar_conv(input)
    x3 = self.srm_conv(input)

    x = [x1,x2,x3]
    x = K.concatenate(x, axis=-1)
    
    x = self.conv2(x)
    x = self.conv3(x)
    x = self.conv4(x)
    #Dropout(0.3)
    x = self.conv5(x)
    x = self.conv6(x)
    x = self.conv7(x)
    #Dropout(0.2)
    x = self.conv8(x)
    x = self.conv9(x)
    x = self.conv10(x)
    #Dropout(0.3)
    x = self.conv11(x)
    x = self.conv12(x)
    x = self.conv13(x)
    #Dropout(0.2)

    x = Lambda(lambda t : K.l2_normalize( t, axis=-1), name='L2')(x)

    x = self.conv14(x)
    x = self.bn(x)

    #new
    #x = self.conv15(x)
    #x = self.conv14(x)
    #x = self.bn2(x)

    #ZPool + LSTM layers 
    zpool_windows = self.ZPool(x, globalPool=True)
    sigma = self.globalstd(x)
    sigma = Lambda( lambda t : K.expand_dims( t, axis=1 ), name='sigma')( sigma )
    zf = Lambda( lambda vs : K.abs(vs[0]/vs[1]), name='ZPool' )([zpool_windows, sigma])
    x = self.conv_lstm(zf)
    
    output = self.conv_last(x)

    return output

#Second implementation of the model
Here, as suggested in the paper, the model is divided in two sub-model, the first one inspired to VGG network, useful for Image Manipulation Classification (IMC) and used to Image manipulation feature extraction, the second one include a ZPool layer inspired to a human reasoning to local anomaly detection.

P.S.: The layers and the structure of this 2 sub-models and the previous  all-in-one model are equal.

In [ ]:
class IMC(Model):

  def __init__(self, classification, k):
    super(IMC, self).__init__()

    self.classification = classification
    self.k = k

    self.conv1 = Conv2D(10, (5,5), input_shape=(IMG_SIZE, IMG_SIZE, 3), padding=pad, activation="relu")
    self.bayar_conv = BayarConv2D(3, (5,5), activation=activations.relu)
    self.srm_conv = SRMConv2D(activation=activations.relu)

    self.conv2 = Conv2D(32, (3,3), padding=pad, activation="relu")
    self.conv3 = Conv2D(64, (3,3), padding=pad, activation="relu")
    self.conv4 = Conv2D(64, (3,3), padding=pad, activation="relu")
    self.conv5 = Conv2D(128, (3,3), padding=pad, activation="relu")
    self.conv6 = Conv2D(128, (3,3), padding=pad, activation="relu")
    self.conv7 = Conv2D(128, (3,3), padding=pad, activation="relu")
    self.conv8 = Conv2D(256, (3,3), padding=pad, activation="relu")
    self.conv9 = Conv2D(256, (3,3), padding=pad, activation="relu")
    self.conv10 = Conv2D(256, (3,3), padding=pad, activation="relu")
    self.conv11 = Conv2D(256, (3,3), padding=pad, activation="relu")
    self.conv12 = Conv2D(256, (3,3), padding=pad, activation="relu")
    self.conv13 = Conv2D(64, (3,3), padding=pad, activation=None, activity_regularizer=regularizers.l2(1e-5))

    self.flatten = Flatten()
    self.dense1 = Dense(64, activation="relu")
    self.dense2 = Dense(self.k, activation="softmax")

  def set_classification(self, classification):
    self.classification = classification

  def call(self, input):
    
    x1 = self.conv1(input)
    x2 = self.bayar_conv(input)
    x3 = self.srm_conv(input)

    x = [x1,x2,x3]
    x = K.concatenate(x, axis=-1)
    
    x = self.conv2(x)
    x = self.conv3(x)
    x = self.conv4(x)
    x = self.conv5(x)
    x = self.conv6(x)
    x = self.conv7(x)
    x = self.conv8(x)
    x = self.conv9(x)
    x = self.conv10(x)
    x = self.conv11(x)
    x = self.conv12(x)
    x = self.conv13(x)
    x = Lambda(lambda t : K.l2_normalize( t, axis=-1), name='L2')(x)

    if self.classification:
      x = self.flatten(x)
      x = self.dense1(x)
      x = self.dense2(x)

    return x

In [ ]:
def mantranet_model(first_block, pool_size_list):

    img = Input(shape=(None,None,3), name='imput')
    out = first_block(img)
    out = Conv2D(64, (1,1), activation=None, use_bias=False, kernel_constraint = unit_norm( axis=-2 ), padding=pad)(out)
    out = BatchNormalization(axis=-1, center=True, scale=True, name="bn1")(out)

    sigma = GlobalStd2D(name='glbStd')(out)
    sigma = Lambda( lambda t : K.expand_dims( t, axis=1 ), name='sigma')( sigma )
    zpool_windows = ZPool2D(pool_size_list)(out, globalPool = True)
    zf = Lambda(lambda vs : K.abs(vs[0]/vs[1]), name='ZPool')([zpool_windows, sigma])

    out = ConvLSTM2D(8, (7,7), activation='tanh', recurrent_activation='hard_sigmoid',
                     padding='same', name='cLSTM', return_sequences=False)(zf)

    output = Conv2D(1, (7,7), padding=pad, activation="sigmoid")(out)

    return Model(inputs=img, outputs=output, name='mantranet')

#Build and compile models

In [ ]:
#BUILD AND COMPILE A NEW (ALL-IN-ONE) MODEL

mantranet = ManTraNet()
mantranet.build([batch_size, IMG_SIZE, IMG_SIZE, 3])
mantranet.summary()
mantranet.compile(optimizer=opt, loss=loss_f, metrics=metrics)

In [ ]:
#BUILD AND COMPILE A NEW IMC MODEL

imc = IMC(True, 7)
imc.build([batch_size, IMG_SIZE, IMG_SIZE, 3])
imc.summary()
imc.compile(optimizer=opt, loss=loss_f, metrics=metrics)

In [ ]:
#BUILD AND COMPILE A NEW MODEL USING PREVIOUS IMC MODEL

imc.trainable=False
imc.set_classification(False)
mantranet = mantranet_model(imc, [7,15,31])
mantranet = ManTraNet()
mantranet.build([batch_size, IMG_SIZE, IMG_SIZE, 3])
mantranet.summary()
mantranet.compile(optimizer=opt, loss=loss_f, metrics=metrics_imc)

#Cell to load pre-trained weights

In [ ]:
#LOAD PRE-TRAINED WEIGHTS (IF NEEDED)
mantranet.load_weights("/content/drive/MyDrive/Mantra_weights/model_3.h5")

#Train a model

In [ ]:
#TRAIN THE MODEL

ImageFile.LOAD_TRUNCATED_IMAGES = True
History=imc.fit(train_imc, epochs=EPOCHS, batch_size=batch_size, verbose=1, validation_data=val_imc)

In [ ]:
#TRAIN THE MODEL
History=mantranet.fit(x_train, y_train, epochs=EPOCHS, batch_size=batch_size, verbose=1, validation_data=(x_val, y_val))

#Cell to save weights

In [ ]:
#SAVE TRAINED WEIGHTS
mantranet.save_weights("/content/drive/MyDrive/Mantra_weights/model_11.h5")

In [ ]:
imc.save_weights("/content/drive/MyDrive/Mantra_weights/imc.h5")

#Plot evaluation, predictions and history of training of the model

In [ ]:
#TEST OF THE MODEL
print_model_evaluation(mantranet)

In [ ]:
#PLOT ORIGINAL IMAGE WITH GROUND TRUTH AND PREDICTED MASK
plot_samples_predictions(mantranet, imgs_list, mask_list, imgs_train_files, IMG_SIZE, n_samples=10)

In [ ]:
#PLOT TRAINING HISTORY OF THE MODEL
plot_history(History)

In [ ]:
#TEST OF THE MODEL
loss, acc = imc.evaluate(test_imc)

print("Loss: "+np.str(loss))
print("Accuracy: "+np.str(acc))

In [ ]:
#PLOT TRAINING HISTORY OF THE MODEL
plot_history2(History)